In [1]:
import numpy as np
import scipy as sci
import scipy.sparse as sparse
import scipy.sparse.linalg as splinalg
import math
from scipy.sparse.linalg import splu
from scipy.sparse import lil_matrix
from scipy.sparse.linalg import spsolve
from scipy.sparse.linalg import lsqr

tol = 1e-5

In [2]:
def generate_positions(m, n, d):
    pos = np.empty([0, 2])
    i = 0
    for y in range(0, n):
        for x in range(0, m):
            pp = np.array([[x * d, y * d]])
            pos = np.concatenate((pos, pp), axis=0)
    return pos

In [3]:
def generate_struts(m, n):
    mn = m * n
    struts = np.empty([mn, mn])
    for i in range(0, mn):
        for j in range(0, mn):
            struts[i][j] = (i != j)
    return struts

In [4]:
def unit_vector(pos, i, j):
    r_vec = pos[j] - pos[i]
    return r_vec / np.linalg.norm(r_vec)

In [5]:
def num_struts(m, n):
    mn = m * n
    n_struts = mn * (mn - 1) / 2
    return int(n_struts)

In [6]:
def generate_strut_indices(m, n):
    mn = m * n
    n_struts = num_struts(m, n)
    strut_indices = np.empty([mn, mn])
    strut_inverse = np.empty([int(n_struts), 2])
    strut_index = 0
    for i in range(0, mn):
        for j in range(0, mn):
            if (i == j):
                strut_indices[i][j] = -1
            elif i > j:
                strut_indices[i][j] = strut_indices[j][i]
            else:
                strut_inverse[strut_index][0] = i
                strut_inverse[strut_index][1] = j
                strut_indices[i][j] = strut_index
                strut_index += 1
    return strut_indices, strut_inverse

In [7]:
def fill_matrix(m, n, vert_load_val, struts, strut_indices, pos):
    tol = 1e-5
    mn = m * n
    n_struts = num_struts(m, n)
    #coeff_matrix = lil_matrix((2 * mn, n_struts))
    coeff_matrix = lil_matrix((n_struts, n_struts))
    const_vector = np.empty([n_struts, 1])
    for k in range(0, n_struts):
        for kk in range(0, n_struts):
            coeff_matrix[k, kk] = 0.0
    for k in range(0, n_struts):
        const_vector[k] = 0.0
        #for u in range(0, int(n_struts)):
        #    coeff_matrix[k, u] = 0.0
    eq_index = 0
    for i in range (0, mn):
        if (abs(vert_load_val[i]) >= tol):
            const_vector[2 * i + 1] = -vert_load_val[i]
        for j in range(0, mn):
            if (struts[i][j] == False):
                continue
            u_vec = unit_vector(pos, i, j)
            if (abs(u_vec[0]) < tol):
                u_vec[0] = 0.0
            if (abs(u_vec[1]) < tol):
                u_vec[1] = 0.0
            coeff_matrix[eq_index, strut_indices[i][j]] = u_vec[0]
            coeff_matrix[eq_index + 1, strut_indices[i][j]] = u_vec[1]
        eq_index += 2
    return coeff_matrix, const_vector

In [8]:
def compute_solution(m, n, d, sup, loads):
    mn = m * n
    n_struts = num_struts(m, n)
    pos = generate_positions(m, n, d)
    struts = generate_struts(m, n)
    strut_indices, strut_inverse = generate_strut_indices(m, n)
    
    vert_sup = np.empty([mn])
    vert_sup_val = np.empty([mn])
    vert_load_val = np.empty([mn])
    for k in range(0, mn):
        vert_sup[k] = False
        vert_sup_val[k] = 0.0
        vert_load_val[k] = 0.0
    for k in range(0, len(loads)):
        vert_load_val[loads[k][0]] = loads[k][1]
    vert_sup[sup[0]] = 1
    vert_sup[sup[1]] = 1
    supA_pos = None
    supB_pos = None
    supA_index = 0
    supB_index = 0
    for supA_index in range(0, mn):
        if (abs(vert_sup[supA_index]) > 0):
            supA_pos = pos[supA_index]
            break
    for supB_index in range(supA_index + 1, mn):
        if (abs(vert_sup[supB_index]) > 0):
            supB_pos = pos[supB_index]
            break
    supA_val = 0.0
    supB_val = 0.0
    vert_load_sum = 0.0
    for i in range(0, mn):
        if (abs(vert_load_val[i]) == 0):
            continue
        load_pos = pos[i]
        delta_x = supB_pos[0] - load_pos[0]
        supA_val += delta_x * vert_load_val[i]
        vert_load_sum += vert_load_val[i]
    supA_val /= supA_pos[0] - supB_pos[0]
    supB_val = -vert_load_sum - supA_val
    vert_load_val[supA_index] += supA_val
    vert_load_val[supB_index] += supB_val
    
    print("Support reactions are:")
    print("(" + str(supA_pos[0]) + ", " + str(supA_pos[1]) + ") : " + str(supA_val))
    print("(" + str(supB_pos[0]) + ", " + str(supB_pos[1]) + ") : " + str(supB_val), "\n")
    
    coeff_matrix, const_vector = fill_matrix(m, n, vert_load_val, struts, strut_indices, pos)
    coeff_matrix = coeff_matrix.tocsc()
    member_forces = lsqr(coeff_matrix, const_vector)[0]
    
    force_sum = np.empty([mn, 2])
    for k in range(0, mn):
        force_sum[k][0] = 0.0
        force_sum[k][1] = 0.0
    print("Nonzero forces in the members are:")
    nonzero_count = 0
    for k in range(0, n_struts):
        strut_index = strut_inverse[k]
        si0 = int(strut_index[0])
        si1 = int(strut_index[1])
        if abs(member_forces[k]) >= tol:
            nonzero_count += 1
            print("(", si0, ", ", si1, ") : ", member_forces[k])
            
        uij_vec = unit_vector(pos, si0, si1)
        uji_vec = unit_vector(pos, si1, si0)
        force_sum[si0] += uij_vec * member_forces[k]
        force_sum[si1] += uji_vec * member_forces[k]
    print("Members:", nonzero_count, "/", n_struts)
    print("\n")
    print("Forces in each node are:")
    for k in range(0, mn):
        if (abs(force_sum[k][0]) < tol):
            force_sum[k][0] = 0.0
        if (abs(force_sum[k][1]) < tol):
            force_sum[k][1] = 0.0
        print(k, ": (", round(force_sum[k][0], 3), ", ", round(force_sum[k][1], 3), ")")
    

In [9]:
compute_solution(16, 5, 1, [32, 47], [[32, 10], [35, 10], [38, 10], [41, 10], [44, 10], [47, 10]])

Support reactions are:
(0.0, 2.0) : -30.0
(15.0, 2.0) : -30.0 

Nonzero forces in the members are:
( 0 ,  1 ) :  -0.00531778880939
( 0 ,  2 ) :  -0.022413186729
( 0 ,  3 ) :  -0.0411065680949
( 0 ,  4 ) :  -0.055665141809
( 0 ,  5 ) :  -0.0666816028395
( 0 ,  6 ) :  -0.0786308024334
( 0 ,  7 ) :  -0.0896910625239
( 0 ,  8 ) :  -0.0988235201811
( 0 ,  9 ) :  -0.109883780272
( 0 ,  10 ) :  -0.121832979865
( 0 ,  11 ) :  -0.132849440896
( 0 ,  12 ) :  -0.14740801461
( 0 ,  13 ) :  -0.166101395976
( 0 ,  14 ) :  -0.183196793896
( 0 ,  15 ) :  -0.188514582705
( 0 ,  16 ) :  -0.0292829826597
( 0 ,  17 ) :  0.0564349673442
( 0 ,  18 ) :  0.0583060587915
( 0 ,  19 ) :  0.0380668803171
( 0 ,  20 ) :  0.0116181574182
( 0 ,  21 ) :  -0.0019670565318
( 0 ,  22 ) :  -0.0167257766456
( 0 ,  23 ) :  -0.0341424530087
( 0 ,  24 ) :  -0.042798181923
( 0 ,  25 ) :  -0.0547511471184
( 0 ,  26 ) :  -0.0707486526591
( 0 ,  27 ) :  -0.0791814872784
( 0 ,  28 ) :  -0.0925693937226
( 0 ,  29 ) :  -0.1153501028

( 8 ,  79 ) :  -0.111268799979
( 9 ,  10 ) :  -0.0119491995939
( 9 ,  11 ) :  -0.0229656606244
( 9 ,  12 ) :  -0.0375242343385
( 9 ,  13 ) :  -0.0562176157043
( 9 ,  14 ) :  -0.073313013624
( 9 ,  15 ) :  -0.0786308024334
( 9 ,  16 ) :  -0.114143107801
( 9 ,  17 ) :  -0.100105992182
( 9 ,  18 ) :  -0.0762153125515
( 9 ,  19 ) :  -0.0513860242996
( 9 ,  20 ) :  -0.0393658066935
( 9 ,  21 ) :  -0.0312825829684
( 9 ,  22 ) :  -0.0125720930204
( 9 ,  23 ) :  -0.00376090418038
( 9 ,  24 ) :  0.00642807292555
( 9 ,  25 ) :  0.0293794023343
( 9 ,  26 ) :  -0.00347134316861
( 9 ,  27 ) :  -0.0173534980748
( 9 ,  28 ) :  -0.0299092889843
( 9 ,  29 ) :  -0.060034061533
( 9 ,  30 ) :  -0.0879635600497
( 9 ,  31 ) :  -0.105426968167
( 9 ,  32 ) :  -0.47672021375
( 9 ,  33 ) :  -0.0660336845411
( 9 ,  34 ) :  -0.0535887538301
( 9 ,  35 ) :  0.16349909124
( 9 ,  36 ) :  -0.0301301854638
( 9 ,  37 ) :  -0.0195030585125
( 9 ,  38 ) :  0.330770384624
( 9 ,  39 ) :  0.000248410680759
( 9 ,  40 ) :  0.00

( 15 ,  31 ) :  -0.0292829826597
( 15 ,  32 ) :  -0.323043741809
( 15 ,  33 ) :  -0.0678322289373
( 15 ,  34 ) :  -0.0552169878113
( 15 ,  35 ) :  0.0638324921859
( 15 ,  36 ) :  -0.0305114584523
( 15 ,  37 ) :  -0.0183821603194
( 15 ,  38 ) :  0.12701264481
( 15 ,  39 ) :  0.00699988824938
( 15 ,  40 ) :  0.0208760377114
( 15 ,  41 ) :  0.229861314494
( 15 ,  42 ) :  0.0527048806464
( 15 ,  43 ) :  0.0712573514566
( 15 ,  44 ) :  0.450658243747
( 15 ,  45 ) :  0.109800998234
( 15 ,  46 ) :  0.118982163766
( 15 ,  47 ) :  -1.73733975269
( 15 ,  48 ) :  -0.0465502588949
( 15 ,  49 ) :  -0.0158939976465
( 15 ,  50 ) :  0.000762576015871
( 15 ,  51 ) :  0.0125885179983
( 15 ,  52 ) :  0.0193350332668
( 15 ,  53 ) :  0.0349252259152
( 15 ,  54 ) :  0.050209557532
( 15 ,  55 ) :  0.0576959151427
( 15 ,  56 ) :  0.0758744143775
( 15 ,  57 ) :  0.0965712498492
( 15 ,  58 ) :  0.103965508198
( 15 ,  59 ) :  0.12351287472
( 15 ,  60 ) :  0.145494634867
( 15 ,  61 ) :  0.124662433431
( 15 ,  62 

( 21 ,  48 ) :  -0.0906511449342
( 21 ,  49 ) :  -0.0573294858209
( 21 ,  50 ) :  -0.0348057766229
( 21 ,  51 ) :  -0.0125816157242
( 21 ,  52 ) :  -0.0119815950549
( 21 ,  54 ) :  0.0207827761452
( 21 ,  55 ) :  0.0094590873722
( 21 ,  56 ) :  0.0040393611502
( 21 ,  57 ) :  0.00841341723325
( 21 ,  59 ) :  -0.00733981088301
( 21 ,  60 ) :  -0.00611119780075
( 21 ,  61 ) :  -0.0114918072145
( 21 ,  62 ) :  -0.0213073436734
( 21 ,  63 ) :  -0.0445832023631
( 21 ,  64 ) :  -0.103965508198
( 21 ,  65 ) :  -0.0864134145255
( 21 ,  66 ) :  -0.063296464447
( 21 ,  67 ) :  -0.0435737191296
( 21 ,  68 ) :  -0.0313445712278
( 21 ,  69 ) :  -0.0182790069675
( 21 ,  70 ) :  -0.00482953540219
( 21 ,  71 ) :  -0.000609532884728
( 21 ,  72 ) :  0.0012539829758
( 21 ,  73 ) :  0.00338407680949
( 21 ,  74 ) :  0.00111048888496
( 21 ,  75 ) :  -0.00199042693549
( 21 ,  76 ) :  -0.0038305826344
( 21 ,  77 ) :  -0.00910220855266
( 21 ,  78 ) :  -0.0196526825887
( 21 ,  79 ) :  -0.0349252259152
( 22 ,  2

( 29 ,  45 ) :  0.0103187841603
( 29 ,  46 ) :  -0.01359812757
( 29 ,  47 ) :  -0.848314994994
( 29 ,  48 ) :  -0.0281702233897
( 29 ,  49 ) :  -0.00742257273793
( 29 ,  51 ) :  0.00320053849223
( 29 ,  52 ) :  0.00329942268636
( 29 ,  53 ) :  0.0114918072145
( 29 ,  54 ) :  0.0183003269804
( 29 ,  55 ) :  0.0180733530683
( 29 ,  56 ) :  0.0273474311858
( 29 ,  57 ) :  0.0382651910577
( 29 ,  58 ) :  0.0348057766229
( 29 ,  59 ) :  0.0412335202463
( 29 ,  60 ) :  0.0480943746541
( 29 ,  62 ) :  -0.0636259190748
( 29 ,  63 ) :  -0.124469627789
( 29 ,  64 ) :  -0.000762576015871
( 29 ,  65 ) :  0.0142758786388
( 29 ,  66 ) :  0.0226711609834
( 29 ,  67 ) :  0.0263909146217
( 29 ,  68 ) :  0.028691987669
( 29 ,  69 ) :  0.0331807004352
( 29 ,  70 ) :  0.0371092366834
( 29 ,  71 ) :  0.0391097951332
( 29 ,  72 ) :  0.0432113198229
( 29 ,  73 ) :  0.0460459274023
( 29 ,  74 ) :  0.0422706711193
( 29 ,  75 ) :  0.0350927167219
( 29 ,  76 ) :  0.0156177769325
( 29 ,  77 ) :  -0.0312024513026


( 40 ,  63 ) :  -0.00943014027406
( 40 ,  64 ) :  -0.00699988824938
( 40 ,  65 ) :  0.00194007497175
( 40 ,  66 ) :  0.00558457810881
( 40 ,  67 ) :  0.00507498298718
( 40 ,  68 ) :  0.0022251087955
( 40 ,  69 ) :  0.00104520188802
( 40 ,  70 ) :  -0.000248410680759
( 40 ,  71 ) :  -0.00572141202339
( 40 ,  72 ) :  -0.00867984752008
( 40 ,  73 ) :  -0.00730259571307
( 40 ,  74 ) :  -0.00841710196604
( 40 ,  75 ) :  -0.00794362867346
( 40 ,  76 ) :  -0.00577377426313
( 40 ,  77 ) :  -0.00678236093817
( 40 ,  78 ) :  -0.0118293761009
( 40 ,  79 ) :  -0.0208760377114
( 41 ,  48 ) :  -0.0624119657132
( 41 ,  49 ) :  -0.0586647099418
( 41 ,  50 ) :  -0.0695213904351
( 41 ,  51 ) :  -0.0869930636575
( 41 ,  52 ) :  -0.111239514273
( 41 ,  53 ) :  -0.13672073208
( 41 ,  54 ) :  -0.179651756962
( 41 ,  55 ) :  -0.265824996444
( 41 ,  56 ) :  -0.421631212165
( 41 ,  57 ) :  -0.590365122129
( 41 ,  58 ) :  -0.431090299537
( 41 ,  59 ) :  -0.277160825573
( 41 ,  60 ) :  -0.191794434177
( 41 ,  61

( 54 ,  59 ) :  0.0249314590663
( 54 ,  60 ) :  0.0332018303362
( 54 ,  61 ) :  0.0472539576286
( 54 ,  62 ) :  0.0592214768223
( 54 ,  63 ) :  0.0600605470165
( 54 ,  64 ) :  0.0167257766456
( 54 ,  65 ) :  0.0176456770325
( 54 ,  66 ) :  0.0112602752561
( 54 ,  67 ) :  0.00144901234888
( 54 ,  68 ) :  -0.0104038441687
( 54 ,  69 ) :  -0.0231393497517
( 54 ,  70 ) :  -0.0293794023343
( 54 ,  71 ) :  -0.0135968779371
( 54 ,  72 ) :  0.000844604075493
( 54 ,  73 ) :  0.0125720930204
( 54 ,  74 ) :  0.0211241392611
( 54 ,  75 ) :  0.0283278597456
( 54 ,  76 ) :  0.0379438620912
( 54 ,  77 ) :  0.0485876207594
( 54 ,  78 ) :  0.0562173160461
( 54 ,  79 ) :  0.0547511471184
( 55 ,  56 ) :  0.00124921093088
( 55 ,  57 ) :  0.00804340029105
( 55 ,  58 ) :  0.0159378483443
( 55 ,  59 ) :  0.0181372697061
( 55 ,  60 ) :  0.026407640976
( 55 ,  61 ) :  0.0404597682684
( 55 ,  62 ) :  0.0524272874621
( 55 ,  63 ) :  0.0532663576563
( 55 ,  64 ) :  0.0341424530087
( 55 ,  65 ) :  0.0357647278327
